<a href="https://colab.research.google.com/github/davicn/poc_vqa/blob/main/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers --quiet
!pip install -U "huggingface_hub[cli]" --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.6/388.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import (BlipProcessor, BlipForQuestionAnswering)
from torch.utils.data import Dataset, DataLoader, Subset
import torch
from huggingface_hub import notebook_login

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

## Dataset

In [ ]:

dataset = load_dataset("SaffalPoosh/deepFashion-with-masks", split='train')

Generating train split:   0%|          | 0/40658 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['images', 'gender', 'pose', 'cloth_type', 'pid', 'caption', 'mask', 'mask_overlay'],
    num_rows: 40658
})

In [ ]:
class CreateDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["images"], text=item["caption"], padding="max_length", return_tensors="pt")
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        return encoding


In [ ]:
train_dataset = CreateDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2)

## Train Model

In [ ]:

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

device = "cuda"
model.to(device)

model.train()

for epoch in range(10):
  print("Epoch:", epoch)

  for idx, batch in enumerate(train_dataloader):
    input_ids = batch.pop("input_ids").to(device)
    pixel_values = batch.pop("pixel_values").to(device)

    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)

    loss = outputs.loss

    print("Loss:", loss.item())

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

    if idx > 10000:
        break

In [ ]:
notebook_login()

In [ ]:
model.push_to_hub('davicn7/hvar_vaq_model')

README.md:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/davicn7/hvar_vaq_model/commit/397738449725829aad8d597d7eb5653efb17ac4c', commit_message='Upload BlipForQuestionAnswering', commit_description='', oid='397738449725829aad8d597d7eb5653efb17ac4c', pr_url=None, pr_revision=None, pr_num=None)